In [ ]:
import micropip
await micropip.install(['ipywidgets', 'matplotlib', 'numpy'])

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

def plot_combined(epsilon):
    """
    核心绘图回调函数：根据传入的相位 epsilon 实时更新图像
    """
    # 1. 初始化画布：创建一个包含两个子图(1行2列)的图形窗口
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
    
    # ==========================================
    # 左子图：时域波形分析 (Time Domain)
    # ==========================================
    
    # 准备数据：定义 x 轴范围 (-4π 到 4π)
    x = np.linspace(-4 * np.pi, 4 * np.pi, 1000)
    # 计算带有相位偏移的 y 值
    y_shifted = np.cos(x + epsilon)
    
    # 绘制参考波形 (虚线，相位为0)
    # [Translate] Label updated to English
    ax1.plot(x, np.cos(x), 'k--', linewidth=2, alpha=0.3, label='y = cos(x) [Ref]')
    # 绘制当前波形 (实线，带有相位 ε)
    ax1.plot(x, y_shifted, 'b-', linewidth=2, label=f'y = cos(x + {epsilon:.2f})')
    
    # 视觉辅助：标记 x=0 处的瞬时值 (红点)
    current_val_at_0 = np.cos(epsilon)
    ax1.plot(0, current_val_at_0, 'ro', markersize=6, zorder=10)
    # 添加数值标签
    ax1.text(0.3, current_val_at_0, f'{current_val_at_0:.2f}', color='red')
    # 添加从原点到该值的红色连线 (增强视觉对比)
    ax1.plot([0, 0], [0, current_val_at_0], color='red', linewidth=2, zorder=10)
    
    # 左图美化设置
    # [Translate] Title updated to English
    ax1.set_title(rf'Time Domain: $y = \cos(x + {epsilon:.2f})$')
    ax1.axhline(0, color='black', linewidth=0.5) # x轴基准线
    ax1.axvline(0, color='black', linewidth=0.5) # y轴基准线
    ax1.set_xlim(-2 * np.pi, 2 * np.pi)
    ax1.set_ylim(-1.5, 1.5)
    ax1.grid(True, linestyle=':', alpha=0.6)
    ax1.legend(loc='upper right')
    
    # ==========================================
    # 右子图：复平面旋转矢量 (Complex Plane)
    # ==========================================
    
    # 绘制背景单位圆 (虚线)
    theta = np.linspace(0, 2*np.pi, 200)
    ax2.plot(np.cos(theta), np.sin(theta), 'k--', alpha=0.3, linewidth=2)
    
    # 根据欧拉公式计算复数坐标: e^(iε) = cos(ε) + i*sin(ε)
    real_part = np.cos(epsilon) # 实部 (x轴)
    imag_part = np.sin(epsilon) # 虚部 (y轴)
    
    # 绘制旋转矢量 (蓝色指针)
    ax2.plot([0, real_part], [0, imag_part], 'b-', linewidth=2, zorder=10)
    
    # 绘制实轴投影点 (红点，对应左图 x=0 处的高度)
    ax2.plot(real_part, 0, 'ro', markersize=6, zorder=8) 
    # 添加投影数值标签
    ax2.text(real_part - 0.05, -0.15, f'{real_part:.2f}', color='red')
    
    # 绘制实轴投影线 (红色粗线，代表 cos(ε) 的长度)
    ax2.plot([0, real_part], [0, 0], color='red', linewidth=2, zorder=8)
    
    # 绘制辅助垂线 (虚线，从圆周点垂直落下到实轴)
    ax2.plot([real_part, real_part], [0, imag_part], 'k--', alpha=0.3 , linewidth=2)
    
    # 绘制角度扇形指示器 (仅当角度不为0时显示)
    if abs(epsilon) > 0.01: 
        arc_angles = np.linspace(0, epsilon, 50)
        ax2.plot(0.3*np.cos(arc_angles), 0.3*np.sin(arc_angles), 'g-', alpha=0.5)
        ax2.text(0.4*np.cos(epsilon/2), 0.4*np.sin(epsilon/2), r'$\varepsilon$', color='green', fontsize=12)

    # 右图美化设置
    # [Translate] Title updated to English
    ax2.set_title(rf'Complex Plane: $e^{{i \cdot {epsilon:.2f}}}$')
    ax2.axhline(0, color='black', linewidth=0.5)
    ax2.axvline(0, color='black', linewidth=0.5)
    ax2.set_aspect('equal') # 强制保持 1:1 比例，确保圆是圆的
    ax2.set_xlim(-1.5, 1.5)
    ax2.set_ylim(-1.5, 1.5)
    ax2.grid(True, linestyle=':', alpha=0.6)

    # 自动调整布局并渲染
    plt.tight_layout()
    plt.show()

# ====================
# 交互组件配置
# ====================

# 1. 创建主控制滑块
slider = widgets.FloatSlider(
    value=0,
    min=-2*np.pi,
    max=2*np.pi,
    step=0.01,
    description='Phase ε:', # [Translate] Updated to English
    layout=widgets.Layout(width='80%')
)

# 2. 创建控制按钮
# 定义不同步长的调节按钮
# [Translate] Kept numbers, but strictly speaking they are universal.
btn_minus_01 = widgets.Button(description='0.1', icon='minus', layout=widgets.Layout(width='80px'))  # 粗调减
btn_minus = widgets.Button(description='0.01', icon='minus', layout=widgets.Layout(width='80px'))    # 微调减
btn_reset = widgets.Button(description='Reset', icon='refresh', layout=widgets.Layout(width='100px')) # [Translate] Reset
btn_plus = widgets.Button(description='0.01', icon='plus', layout=widgets.Layout(width='80px'))      # 微调加
btn_plus_01 = widgets.Button(description='0.1', icon='plus', layout=widgets.Layout(width='80px'))    # 粗调加

# 3. 定义按钮点击的回调函数
def on_minus_01_click(b):
    """减少 0.1"""
    new_val = slider.value - 0.1
    if new_val >= slider.min:
        slider.value = round(new_val, 2) # 使用 round 防止浮点数精度误差

def on_minus_click(b):
    """减少 0.01"""
    new_val = slider.value - 0.01
    if new_val >= slider.min:
        slider.value = round(new_val, 2)

def on_reset_click(b):
    """重置为 0"""
    slider.value = 0.0

def on_plus_click(b):
    """增加 0.01"""
    new_val = slider.value + 0.01
    if new_val <= slider.max:
        slider.value = round(new_val, 2)

def on_plus_01_click(b):
    """增加 0.1"""
    new_val = slider.value + 0.1
    if new_val <= slider.max:
        slider.value = round(new_val, 2)

# 4. 绑定事件处理
# 将按钮的点击事件与上述函数连接
btn_minus_01.on_click(on_minus_01_click)
btn_minus.on_click(on_minus_click)
btn_reset.on_click(on_reset_click)
btn_plus.on_click(on_plus_click)
btn_plus_01.on_click(on_plus_01_click)

# 5. 建立输出关联
# interactive_output 会监听滑块值的变化，并自动调用 plot_combined 函数重绘图像
out = widgets.interactive_output(plot_combined, {'epsilon': slider})

# 6. 组装界面布局
# VBox: 垂直排列 (滑块 -> 按钮组 -> 图像)
# HBox: 水平排列 (按钮横向排布)
ui = widgets.VBox([
    slider, 
    widgets.HBox([btn_minus_01, btn_minus, btn_reset, btn_plus, btn_plus_01]),
    out
])

# 7. 在 Notebook 中显示最终界面
display(ui)